# Export land-unit level variables
- This script is used to export CUHI, SUHI, SWBGT, TBUILD variables at the landunit level;
- Simulations: CNTL, ROOF_TV, IMPRAOD_TV, WALL_TV, ROOF_IMPROAD_TV, ROOF_IMPRAOD_WALL_TV

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
path = '/work/n02/n02/yuansun/cesm/'
# surface
sfile = path + 'cesm_inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr1850_c190214.nc'
#case0
path_0 = path + 'archive/case0/lnd/hist/'
#case2
path_2 = path + 'archive/case2/lnd/hist/'
#case3
path_3 = path + 'archive/case3/lnd/hist/'
#case5
path_5 = path + 'archive/case5/lnd/hist/'

In [3]:
ds_0 = xr.open_dataset(sfile)
mask = np.any(ds_0['PCT_URBAN'] != 0, axis=0)
mask = mask.rename({'lsmlat': 'lat', 'lsmlon': 'lon'})

urban = ds_0['PCT_URBAN'] 
mask_tbd = urban[0,:,:] != 0
mask_hd = urban[1,:,:] != 0
mask_md = urban[2,:,:] != 0

mask_tbd  = mask_tbd .rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
mask_hd  = mask_hd .rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
mask_md  = mask_md .rename({'lsmlat': 'lat', 'lsmlon': 'lon'})  # <xarray.DataArray 'PCT_URBAN' (lsmlat: 192, lsmlon: 288)>

In [4]:
start_year = 2015
end_year = 2100

In [5]:
# case0
Arr_UHImax_tbd_0 = []
Arr_UHImax_hd_0 = []
Arr_UHImax_md_0 = []
Arr_UHImin_tbd_0 = []
Arr_UHImin_hd_0 = []
Arr_UHImin_md_0 = []
Arr_TBUILD_tbd_0 = []
Arr_TBUILD_hd_0 = []
Arr_TBUILD_md_0 = []
Arr_SWBGT_U_tbd_0 = []
Arr_SWBGT_U_hd_0 = []
Arr_SWBGT_U_md_0 = []

for year in range(start_year,end_year):
    fn = 'case0.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    fn1 = 'case0.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_0+fn)):
        ds = xr.open_dataset(path_0+fn)
        ds1 = xr.open_dataset(path_0+fn1)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        TMAX_U_0 = ds['TREFMXAV_U']
        TMIN_U_0 = ds['TREFMNAV_U']
        SWBGT_U_0 = ds['SWBGT_U']
        TBUILD_0 = ds['TBUILD']-273.15
        
        grid_TMAX_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMAX_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMAX_U_0.values
        grid_TMAX_U_dims_0 = xr.DataArray(grid_TMAX_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMAX_U_dims_0 = grid_TMAX_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMAX_U_tbd_0 = (grid_TMAX_U_dims_0.sel(lndtype=7)-ds1['TREFMXAV_R']).where(mask_tbd).mean().item()
        TMAX_U_hd_0 = (grid_TMAX_U_dims_0.sel(lndtype=8)-ds1['TREFMXAV_R']).where(mask_hd).mean().item()
        TMAX_U_md_0 = (grid_TMAX_U_dims_0.sel(lndtype=9)-ds1['TREFMXAV_R']).where(mask_md).mean().item()
        
        Arr_UHImax_tbd_0.append(TMAX_U_tbd_0)
        Arr_UHImax_hd_0.append(TMAX_U_hd_0)
        Arr_UHImax_md_0.append(TMAX_U_md_0)

        grid_TMIN_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMIN_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMIN_U_0.values
        grid_TMIN_U_dims_0 = xr.DataArray(grid_TMIN_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMIN_U_dims_0 = grid_TMIN_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMIN_U_tbd_0 = (grid_TMIN_U_dims_0.sel(lndtype=7)-ds1['TREFMNAV_R']).where(mask_tbd).mean().item()
        TMIN_U_hd_0 = (grid_TMIN_U_dims_0.sel(lndtype=8)-ds1['TREFMNAV_R']).where(mask_hd).mean().item()
        TMIN_U_md_0 = (grid_TMIN_U_dims_0.sel(lndtype=9)-ds1['TREFMNAV_R']).where(mask_md).mean().item()
        
        Arr_UHImin_tbd_0.append(TMIN_U_tbd_0)
        Arr_UHImin_hd_0.append(TMIN_U_hd_0)
        Arr_UHImin_md_0.append(TMIN_U_md_0) 
        
        grid_TBUILD_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TBUILD_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TBUILD_0.values
        grid_TBUILD_dims_0 = xr.DataArray(grid_TBUILD_0, dims=("time","lndtype","lat","lon"))
        grid_TBUILD_dims_0 = grid_TBUILD_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TBUILD_tbd_0 = grid_TBUILD_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        TBUILD_hd_0 = grid_TBUILD_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        TBUILD_md_0 = grid_TBUILD_dims_0.sel(lndtype=9).where(mask_md).mean().item()
        Arr_TBUILD_tbd_0.append(TBUILD_tbd_0)
        Arr_TBUILD_hd_0.append(TBUILD_hd_0)
        Arr_TBUILD_md_0.append(TBUILD_md_0)
        
        grid_SWBGT_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_SWBGT_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = SWBGT_U_0.values
        grid_SWBGT_U_dims_0 = xr.DataArray(grid_SWBGT_U_0, dims=("time","lndtype","lat","lon"))
        grid_SWBGT_U_dims_0 = grid_SWBGT_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        SWBGT_U_tbd_0 = grid_SWBGT_U_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        SWBGT_U_hd_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        SWBGT_U_md_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_md).mean().item()
        Arr_SWBGT_U_tbd_0.append(SWBGT_U_tbd_0)
        Arr_SWBGT_U_hd_0.append(SWBGT_U_hd_0)
        Arr_SWBGT_U_md_0.append(SWBGT_U_md_0)

In [6]:
data_dict_0 = {
    'UHImax_tbd_0':Arr_UHImax_tbd_0,
    'UHImax_hd_0':Arr_UHImax_hd_0,
    'UHImax_md_0':Arr_UHImax_md_0,
    'UHImin_tbd_0':Arr_UHImin_tbd_0,
    'UHImin_hd_0':Arr_UHImin_hd_0,
    'UHImin_md_0':Arr_UHImin_md_0,
    'TBUILD_tbd_0': Arr_TBUILD_tbd_0,
    'TBUILD_hd_0':Arr_TBUILD_hd_0,
    'TBUILD_md_0':Arr_TBUILD_md_0,
    'SWBGT_U_tbd_0': Arr_SWBGT_U_tbd_0,
    'SWBGT_U_hd_0':Arr_SWBGT_U_hd_0,
    'SWBGT_U_md_0':Arr_SWBGT_U_md_0
}
pd.DataFrame(data_dict_0).to_csv('mon_0.csv', index=False)

In [7]:
# case2
Arr_UHImax_tbd_2 = []
Arr_UHImax_hd_2 = []
Arr_UHImax_md_2 = []
Arr_UHImin_tbd_2 = []
Arr_UHImin_hd_2 = []
Arr_UHImin_md_2 = []
Arr_TBUILD_tbd_2 = []
Arr_TBUILD_hd_2 = []
Arr_TBUILD_md_2 = []
Arr_SWBGT_U_tbd_2 = []
Arr_SWBGT_U_hd_2 = []
Arr_SWBGT_U_md_2 = []

for year in range(start_year,end_year):
    fn = 'case2.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    fn1 = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        ds1 = xr.open_dataset(path_2+fn1)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        TMAX_U_0 = ds['TREFMXAV_U']
        TMIN_U_0 = ds['TREFMNAV_U']
        SWBGT_U_0 = ds['SWBGT_U']
        TBUILD_0 = ds['TBUILD']-273.15
        
        grid_TMAX_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMAX_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMAX_U_0.values
        grid_TMAX_U_dims_0 = xr.DataArray(grid_TMAX_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMAX_U_dims_0 = grid_TMAX_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMAX_U_tbd_0 = (grid_TMAX_U_dims_0.sel(lndtype=7)-ds1['TREFMXAV_R']).where(mask_tbd).mean().item()
        TMAX_U_hd_0 = (grid_TMAX_U_dims_0.sel(lndtype=8)-ds1['TREFMXAV_R']).where(mask_hd).mean().item()
        TMAX_U_md_0 = (grid_TMAX_U_dims_0.sel(lndtype=9)-ds1['TREFMXAV_R']).where(mask_md).mean().item()
        
        Arr_UHImax_tbd_2.append(TMAX_U_tbd_0)
        Arr_UHImax_hd_2.append(TMAX_U_hd_0)
        Arr_UHImax_md_2.append(TMAX_U_md_0)

        grid_TMIN_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMIN_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMIN_U_0.values
        grid_TMIN_U_dims_0 = xr.DataArray(grid_TMIN_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMIN_U_dims_0 = grid_TMIN_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMIN_U_tbd_0 = (grid_TMIN_U_dims_0.sel(lndtype=7)-ds1['TREFMNAV_R']).where(mask_tbd).mean().item()
        TMIN_U_hd_0 = (grid_TMIN_U_dims_0.sel(lndtype=8)-ds1['TREFMNAV_R']).where(mask_hd).mean().item()
        TMIN_U_md_0 = (grid_TMIN_U_dims_0.sel(lndtype=9)-ds1['TREFMNAV_R']).where(mask_md).mean().item()
        
        Arr_UHImin_tbd_2.append(TMIN_U_tbd_0)
        Arr_UHImin_hd_2.append(TMIN_U_hd_0)
        Arr_UHImin_md_2.append(TMIN_U_md_0)
        
        grid_TBUILD_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TBUILD_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TBUILD_0.values
        grid_TBUILD_dims_0 = xr.DataArray(grid_TBUILD_0, dims=("time","lndtype","lat","lon"))
        grid_TBUILD_dims_0 = grid_TBUILD_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TBUILD_tbd_0 = grid_TBUILD_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        TBUILD_hd_0 = grid_TBUILD_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        TBUILD_md_0 = grid_TBUILD_dims_0.sel(lndtype=9).where(mask_md).mean().item()
        Arr_TBUILD_tbd_2.append(TBUILD_tbd_0)
        Arr_TBUILD_hd_2.append(TBUILD_hd_0)
        Arr_TBUILD_md_2.append(TBUILD_md_0)
        
        grid_SWBGT_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_SWBGT_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = SWBGT_U_0.values
        grid_SWBGT_U_dims_0 = xr.DataArray(grid_SWBGT_U_0, dims=("time","lndtype","lat","lon"))
        grid_SWBGT_U_dims_0 = grid_SWBGT_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        SWBGT_U_tbd_0 = grid_SWBGT_U_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        SWBGT_U_hd_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        SWBGT_U_md_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_md).mean().item()
        Arr_SWBGT_U_tbd_2.append(SWBGT_U_tbd_0)
        Arr_SWBGT_U_hd_2.append(SWBGT_U_hd_0)
        Arr_SWBGT_U_md_2.append(SWBGT_U_md_0)

In [8]:
data_dict_2 = {
    'UHImax_tbd_2':Arr_UHImax_tbd_2,
    'UHImax_hd_2':Arr_UHImax_hd_2,
    'UHImax_md_2':Arr_UHImax_md_2,
    'UHImin_tbd_2':Arr_UHImin_tbd_2,
    'UHImin_hd_2':Arr_UHImin_hd_2,
    'UHImin_md_2':Arr_UHImin_md_2,
    'TBUILD_tbd_2': Arr_TBUILD_tbd_2,
    'TBUILD_hd_2':Arr_TBUILD_hd_2,
    'TBUILD_md_2':Arr_TBUILD_md_2,
    'SWBGT_U_tbd_2': Arr_SWBGT_U_tbd_2,
    'SWBGT_U_hd_2':Arr_SWBGT_U_hd_2,
    'SWBGT_U_md_2':Arr_SWBGT_U_md_2
}
pd.DataFrame(data_dict_2).to_csv('mon_2.csv', index=False)

In [9]:
# case3
Arr_UHImax_tbd_3 = []
Arr_UHImax_hd_3 = []
Arr_UHImax_md_3 = []
Arr_UHImin_tbd_3 = []
Arr_UHImin_hd_3 = []
Arr_UHImin_md_3 = []
Arr_TBUILD_tbd_3 = []
Arr_TBUILD_hd_3 = []
Arr_TBUILD_md_3 = []
Arr_SWBGT_U_tbd_3 = []
Arr_SWBGT_U_hd_3 = []
Arr_SWBGT_U_md_3 = []

for year in range(start_year,end_year):
    fn = 'case3.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    fn1 = 'case3.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_3+fn)):
        ds = xr.open_dataset(path_3+fn)
        ds1 = xr.open_dataset(path_3+fn1)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        TMAX_U_0 = ds['TREFMXAV_U']
        TMIN_U_0 = ds['TREFMNAV_U']
        SWBGT_U_0 = ds['SWBGT_U']
        TBUILD_0 = ds['TBUILD']-273.15
        
        grid_TMAX_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMAX_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMAX_U_0.values
        grid_TMAX_U_dims_0 = xr.DataArray(grid_TMAX_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMAX_U_dims_0 = grid_TMAX_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMAX_U_tbd_0 = (grid_TMAX_U_dims_0.sel(lndtype=7)-ds1['TREFMXAV_R']).where(mask_tbd).mean().item()
        TMAX_U_hd_0 = (grid_TMAX_U_dims_0.sel(lndtype=8)-ds1['TREFMXAV_R']).where(mask_hd).mean().item()
        TMAX_U_md_0 = (grid_TMAX_U_dims_0.sel(lndtype=9)-ds1['TREFMXAV_R']).where(mask_md).mean().item()
        Arr_UHImax_tbd_3.append(TMAX_U_tbd_0)
        Arr_UHImax_hd_3.append(TMAX_U_hd_0)
        Arr_UHImax_md_3.append(TMAX_U_md_0)

        grid_TMIN_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMIN_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMIN_U_0.values
        grid_TMIN_U_dims_0 = xr.DataArray(grid_TMIN_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMIN_U_dims_0 = grid_TMIN_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMIN_U_tbd_0 = (grid_TMIN_U_dims_0.sel(lndtype=7)-ds1['TREFMNAV_R']).where(mask_tbd).mean().item()
        TMIN_U_hd_0 = (grid_TMIN_U_dims_0.sel(lndtype=8)-ds1['TREFMNAV_R']).where(mask_hd).mean().item()
        TMIN_U_md_0 = (grid_TMIN_U_dims_0.sel(lndtype=9)-ds1['TREFMNAV_R']).where(mask_md).mean().item()
        Arr_UHImin_tbd_3.append(TMIN_U_tbd_0)
        Arr_UHImin_hd_3.append(TMIN_U_hd_0)
        Arr_UHImin_md_3.append(TMIN_U_md_0)
        
        grid_TBUILD_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TBUILD_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TBUILD_0.values
        grid_TBUILD_dims_0 = xr.DataArray(grid_TBUILD_0, dims=("time","lndtype","lat","lon"))
        grid_TBUILD_dims_0 = grid_TBUILD_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TBUILD_tbd_0 = grid_TBUILD_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        TBUILD_hd_0 = grid_TBUILD_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        TBUILD_md_0 = grid_TBUILD_dims_0.sel(lndtype=9).where(mask_md).mean().item()
        Arr_TBUILD_tbd_3.append(TBUILD_tbd_0)
        Arr_TBUILD_hd_3.append(TBUILD_hd_0)
        Arr_TBUILD_md_3.append(TBUILD_md_0)
        
        grid_SWBGT_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_SWBGT_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = SWBGT_U_0.values
        grid_SWBGT_U_dims_0 = xr.DataArray(grid_SWBGT_U_0, dims=("time","lndtype","lat","lon"))
        grid_SWBGT_U_dims_0 = grid_SWBGT_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        SWBGT_U_tbd_0 = grid_SWBGT_U_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        SWBGT_U_hd_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        SWBGT_U_md_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_md).mean().item()
        Arr_SWBGT_U_tbd_3.append(SWBGT_U_tbd_0)
        Arr_SWBGT_U_hd_3.append(SWBGT_U_hd_0)
        Arr_SWBGT_U_md_3.append(SWBGT_U_md_0)

In [10]:
data_dict_3 = {
    'UHImax_tbd_3':Arr_UHImax_tbd_3,
    'UHImax_hd_3':Arr_UHImax_hd_3,
    'UHImax_md_3':Arr_UHImax_md_3,
    'UHImin_tbd_3':Arr_UHImin_tbd_3,
    'UHImin_hd_3':Arr_UHImin_hd_3,
    'UHImin_md_3':Arr_UHImin_md_3,
    'TBUILD_tbd_3': Arr_TBUILD_tbd_3,
    'TBUILD_hd_3':Arr_TBUILD_hd_3,
    'TBUILD_md_3':Arr_TBUILD_md_3,
    'SWBGT_U_tbd_3': Arr_SWBGT_U_tbd_3,
    'SWBGT_U_hd_3':Arr_SWBGT_U_hd_3,
    'SWBGT_U_md_3':Arr_SWBGT_U_md_3
}
pd.DataFrame(data_dict_3).to_csv('mon_3.csv', index=False)

In [11]:
# case5
Arr_UHImax_tbd_5 = []
Arr_UHImax_hd_5 = []
Arr_UHImax_md_5 = []
Arr_UHImin_tbd_5 = []
Arr_UHImin_hd_5 = []
Arr_UHImin_md_5 = []
Arr_TBUILD_tbd_5 = []
Arr_TBUILD_hd_5 = []
Arr_TBUILD_md_5 = []
Arr_SWBGT_U_tbd_5 = []
Arr_SWBGT_U_hd_5 = []
Arr_SWBGT_U_md_5 = []

for year in range(start_year,end_year):
    fn = 'case5.clm2.h0.' + '%04.0f' % year + '-02-01-00000.nc'
    fn1 = 'case5.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_5+fn)):
        ds = xr.open_dataset(path_5+fn)
        ds1 = xr.open_dataset(path_5+fn1)
        lat = ds.lat
        lon = ds.lon
        time = ds.time
        ixy = ds.land1d_ixy
        jxy = ds.land1d_jxy
        landtype = ds.land1d_ityplunit
        nlat = len(lat.values)
        nlon = len(lon.values)
        ntim   = len(time.values)
        nland = int(np.max(landtype))+1
        TMAX_U_0 = ds['TREFMXAV_U']
        TMIN_U_0 = ds['TREFMNAV_U']
        SWBGT_U_0 = ds['SWBGT_U']
        TBUILD_0 = ds['TBUILD']-273.15
        
        grid_TMAX_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMAX_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMAX_U_0.values
        grid_TMAX_U_dims_0 = xr.DataArray(grid_TMAX_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMAX_U_dims_0 = grid_TMAX_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMAX_U_tbd_0 = (grid_TMAX_U_dims_0.sel(lndtype=7)-ds1['TREFMXAV_R']).where(mask_tbd).mean().item()
        TMAX_U_hd_0 = (grid_TMAX_U_dims_0.sel(lndtype=8)-ds1['TREFMXAV_R']).where(mask_hd).mean().item()
        TMAX_U_md_0 = (grid_TMAX_U_dims_0.sel(lndtype=9)-ds1['TREFMXAV_R']).where(mask_md).mean().item()
        Arr_UHImax_tbd_5.append(TMAX_U_tbd_0)
        Arr_UHImax_hd_5.append(TMAX_U_hd_0)
        Arr_UHImax_md_5.append(TMAX_U_md_0)

        grid_TMIN_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TMIN_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TMIN_U_0.values
        grid_TMIN_U_dims_0 = xr.DataArray(grid_TMIN_U_0, dims=("time","lndtype","lat","lon"))
        grid_TMIN_U_dims_0 = grid_TMIN_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TMIN_U_tbd_0 = (grid_TMIN_U_dims_0.sel(lndtype=7)-ds1['TREFMNAV_R']).where(mask_tbd).mean().item()
        TMIN_U_hd_0 = (grid_TMIN_U_dims_0.sel(lndtype=8)-ds1['TREFMNAV_R']).where(mask_hd).mean().item()
        TMIN_U_md_0 = (grid_TMIN_U_dims_0.sel(lndtype=9)-ds1['TREFMNAV_R']).where(mask_md).mean().item()
        Arr_UHImin_tbd_5.append(TMIN_U_tbd_0)
        Arr_UHImin_hd_5.append(TMIN_U_hd_0)
        Arr_UHImin_md_5.append(TMIN_U_md_0)
        
        grid_SWBGT_U_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_SWBGT_U_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = SWBGT_U_0.values
        grid_SWBGT_U_dims_0 = xr.DataArray(grid_SWBGT_U_0, dims=("time","lndtype","lat","lon"))
        grid_SWBGT_U_dims_0 = grid_SWBGT_U_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        SWBGT_U_tbd_0 = grid_SWBGT_U_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        SWBGT_U_hd_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        SWBGT_U_md_0 = grid_SWBGT_U_dims_0.sel(lndtype=8).where(mask_md).mean().item()
        Arr_SWBGT_U_tbd_5.append(SWBGT_U_tbd_0)
        Arr_SWBGT_U_hd_5.append(SWBGT_U_hd_0)
        Arr_SWBGT_U_md_5.append(SWBGT_U_md_0)
        
        grid_TBUILD_0 = np.full([ntim,nland,nlat,nlon],np.nan)
        grid_TBUILD_0[:,landtype.values.astype(int),jxy.values.astype(int) - 1,ixy.values.astype(int) - 1] = TBUILD_0.values
        grid_TBUILD_dims_0 = xr.DataArray(grid_TBUILD_0, dims=("time","lndtype","lat","lon"))
        grid_TBUILD_dims_0 = grid_TBUILD_dims_0.assign_coords(time=ds.time,lndtype=[i for i in range(10)],lat=lat.values,lon=lon.values)
        TBUILD_tbd_0 = grid_TBUILD_dims_0.sel(lndtype=7).where(mask_tbd).mean().item()
        TBUILD_hd_0 = grid_TBUILD_dims_0.sel(lndtype=8).where(mask_hd).mean().item()
        TBUILD_md_0 = grid_TBUILD_dims_0.sel(lndtype=9).where(mask_md).mean().item()
        Arr_TBUILD_tbd_5.append(TBUILD_tbd_0)
        Arr_TBUILD_hd_5.append(TBUILD_hd_0)
        Arr_TBUILD_md_5.append(TBUILD_md_0)

In [12]:
data_dict_5 = {
    'UHImax_tbd_5':Arr_UHImax_tbd_5,
    'UHImax_hd_5':Arr_UHImax_hd_5,
    'UHImax_md_5':Arr_UHImax_md_5,
    'UHImin_tbd_5':Arr_UHImin_tbd_5,
    'UHImin_hd_5':Arr_UHImin_hd_5,
    'UHImin_md_5':Arr_UHImin_md_5,
    'TBUILD_tbd_5': Arr_TBUILD_tbd_5,
    'TBUILD_hd_5':Arr_TBUILD_hd_5,
    'TBUILD_md_5':Arr_TBUILD_md_5,
    'SWBGT_U_tbd_5': Arr_SWBGT_U_tbd_5,
    'SWBGT_U_hd_5':Arr_SWBGT_U_hd_5,
    'SWBGT_U_md_5':Arr_SWBGT_U_md_5
}
pd.DataFrame(data_dict_5).to_csv('mon_5.csv', index=False)